# Week 05
# Lab
## Poisson regression modeling

## Load Libraries

In [1]:
# for reading json files
import json

# numerical libraries
import numpy as np
import scipy as sp
import pystan

# pandas!
import pandas as pd

# plotting libraries
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
sns.set(style="white")

## Make a nice machine-readable dataset

Convert the posted Stata file into a JSON with column definitions and other metadata.

In [3]:
# read in raw stata dataset
nmes_df = pd.read_stata('../../data/nmes.dta')

In [4]:
# rename columns using lower case
col_rename = {}
for s in list(nmes_df.columns):
    col_rename[s] = str.lower(s)

In [5]:
# variable definitions from the paper
var_def = {'ofp':'visits to a physician in an office setting',
           'ofnp':'visits to a non-physician in an office setting',
           'opp':'visits to a physician in a hospital outpatient setting',
           'opnp':'visits to a non-physician in an outpatient setting',
           'emr':'visits to an emergency room',
           'hosp':'number of hospital stays',
           'exclhlth':'perceived excellent health',
           'poorhlth':'perceived poor health',
           'numchron':'number of chronic diseases and conditions',
           'adldiff':'measure of disability status',
           'noreast':'indicator, northeast united stats',
           'midwest':'indicator, midwest united states',
           'west':'indicator, western united states',
           'age':'age',
           'afamn':'indicator, 1 if african american',
           'male':'indicator, 1 if male',
           'married':'indicator, 1 if married',
           'school':'level of education',
           'faminc':'family income',
           'employed':'indicator, 1 if employed',
           'privins':'indicator, 1 if person has private supplementary insurance',
           'medicaid':'indicator, 1 if person has medicaid'
          }

In [6]:
# define the metadata for the dataset
nmes_dict = {'info':'The data are obtained from the National Medical Expenditure Survey (NMES) which was conducted in 1987 and 1988 to provide a comprehensive picture of how Americans use and pay for health services. The NMES is based upon a representative, national probability sample of the civilian, non-institutionalized population and individuals admitted to long-term care facilities during 1987. Under the household survey of the NMES, more than 38000 individuals in 15000 households across the United States were interviewed quarterly about their health insurance coverage, the services they used, and the cost and source of payments of those services. These data were verified by cross-checking information provided by survey respondents with providers of health-care services. In addition to health-care data, NMES provides information on health status, employment, sociodemographic characteristics, and economic status. In this paper we consider a subsample of individuals ages 66 and over (a total of 4406 observations) all of whom are covered by Medicare, a public insurance programme that offers substantial protection against health care costs.',
             'source':'Partha Deb and Pravin K. Trivedi, 1997, Journal of Applied Econometrics, Volume 12, Number 3. Demand for Medical Care by the Elderly: A Finite Mixture Approach',
             'url':'http://cameron.econ.ucdavis.edu/racd2/racd06data1healthcare.dta',
             'vars':var_def
            }

In [7]:
# pandas rename columns
nmes_df.rename(columns=col_rename, inplace=True)
nmes_df.index.name = 'patient'

# and output the dataframe to a dictionary
nmes_dict['data'] = nmes_df.to_dict()

# and output data to a csv
nmes_df.to_csv('../../data/nmes_data.csv')

In [8]:
# write metadata to a dictionary
with open('../../data/nmes_data.json', 'w') as fp:
    json.dump(nmes_dict, fp)

# close the file
fp.close()

## Load Data

### A function to print a long string nicely

In [9]:
def print_info(info,wpl=12):
    """
    nicely print a long paragraph
    """
    
    long_info = info.split()
    num_lines = round(len(long_info) / wpl)
    
    info_break = []
    
    # break up the long string into multiple lines
    for i in range(num_lines):
        hld = ''
        chunk = long_info[wpl*i:wpl*(i+1)]
        
        # piece each line into one string
        for i in range(len(chunk)):
            hld = hld + chunk[i] + ' '
        
        info_break.append(hld)
    
    # now print!
    for i in range(len(info_break)):
        print(info_break[i])

In [10]:
def print_vars(var_dict):
    """
    nicely print the infomation about each variable
    """
    # what's the longest variable name?
    max_len = 0
    for k in var_dict.keys():
        if len(k) > max_len:
            max_len = len(k)
    
    for k in var_dict.keys():
        len_k = len(k)
        print(str(k) + ' '*(max_len - len_k + 1) + ' :::  ' + var_dict[k])

## Read in data

In [11]:
# read json file into a dictionary
with open('../../data/nmes_data.json', 'r') as f:
    json_data = json.load(f)

# close the file
f.close()

In [12]:
# what's the source?
print(json_data['source'])

Partha Deb and Pravin K. Trivedi, 1997, Journal of Applied Econometrics, Volume 12, Number 3. Demand for Medical Care by the Elderly: A Finite Mixture Approach


In [13]:
# where can i get these data?
print(json_data['url'])

http://cameron.econ.ucdavis.edu/racd2/racd06data1healthcare.dta


In [14]:
# print some info about the dataset
print_info(json_data['info'])

The data are obtained from the National Medical Expenditure Survey (NMES) which 
was conducted in 1987 and 1988 to provide a comprehensive picture of 
how Americans use and pay for health services. The NMES is based 
upon a representative, national probability sample of the civilian, non-institutionalized population and 
individuals admitted to long-term care facilities during 1987. Under the household survey 
of the NMES, more than 38000 individuals in 15000 households across the 
United States were interviewed quarterly about their health insurance coverage, the services 
they used, and the cost and source of payments of those services. 
These data were verified by cross-checking information provided by survey respondents with 
providers of health-care services. In addition to health-care data, NMES provides information 
on health status, employment, sociodemographic characteristics, and economic status. In this paper 
we consider a subsample of individuals ages 66 and over (a total 

In [15]:
# what variables are in the dataset?
print_vars(json_data['vars'])

ofp       :::  visits to a physician in an office setting
ofnp      :::  visits to a non-physician in an office setting
opp       :::  visits to a physician in a hospital outpatient setting
opnp      :::  visits to a non-physician in an outpatient setting
emr       :::  visits to an emergency room
hosp      :::  number of hospital stays
exclhlth  :::  perceived excellent health
poorhlth  :::  perceived poor health
numchron  :::  number of chronic diseases and conditions
adldiff   :::  measure of disability status
noreast   :::  indicator, northeast united stats
midwest   :::  indicator, midwest united states
west      :::  indicator, western united states
age       :::  age
afamn     :::  indicator, 1 if african american
male      :::  indicator, 1 if male
married   :::  indicator, 1 if married
school    :::  level of education
faminc    :::  family income
employed  :::  indicator, 1 if employed
privins   :::  indicator, 1 if person has private supplementary insurance
medicaid  :::  in

In [16]:
# just give it to me in a dataframe
data = pd.DataFrame(json_data['data'])
data.head(10)

,ofp,ofnp,opp,opnp,emr,hosp,exclhlth,poorhlth,numchron,adldiff,...,west,age,black,male,married,school,faminc,employed,privins,medicaid
0,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,...,0.0,6.9,1.0,1.0,1.0,6.0,2.8810,1.0,1.0,0.0
1,1.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,...,0.0,7.4,0.0,0.0,1.0,10.0,2.7478,0.0,1.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,6.6,0.0,1.0,1.0,8.0,2.9498,1.0,1.0,0.0
100,9.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,6.6,0.0,1.0,0.0,12.0,3.0900,1.0,1.0,0.0
1000,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,8.1,0.0,0.0,0.0,12.0,0.4600,0.0,1.0,0.0
1001,0.0,0.0,13.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,...,0.0,8.5,1.0,1.0,1.0,14.0,1.5077,0.0,1.0,0.0
1002,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,7.3,1.0,0.0,1.0,8.0,1.5077,0.0,1.0,0.0
1003,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,...,0.0,7.9,1.0,0.0,0.0,4.0,0.4140,0.0,0.0,0.0
1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6.9,1.0,1.0,1.0,15.0,2.0900,0.0,1.0,0.0
1005,16.0,0.0,6.0,2.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,6.9,0.0,1.0,1.0,12.0,3.4583,0.0,1.0,0.0


In [17]:
data.describe()

,ofp,ofnp,opp,opnp,emr,hosp,exclhlth,poorhlth,numchron,adldiff,...,west,age,black,male,married,school,faminc,employed,privins,medicaid
count,4406.000000,4406.000000,4406.000000,4406.000000,4406.000000,4406.000000,4406.000000,4406.000000,4406.000000,4406.000000,...,4406.000000,4406.000000,4406.000000,4406.000000,4406.000000,4406.000000,4406.000000,4406.000000,4406.000000,4406.000000
mean,5.774399,1.618021,0.750794,0.536087,0.263504,0.295960,0.077848,0.125738,1.541988,0.204040,...,0.181117,7.402406,0.117113,0.403541,0.546074,10.290286,2.527132,0.103268,0.776441,0.091239
std,6.759225,5.317056,3.652759,3.879506,0.703659,0.746398,0.267963,0.331591,1.349632,0.403044,...,0.385159,0.633405,0.321591,0.490663,0.497929,3.738736,2.924648,0.304343,0.416677,0.287982
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,6.600000,0.000000,0.000000,0.000000,0.000000,-1.012500,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,6.900000,0.000000,0.000000,0.000000,8.000000,0.912150,0.000000,1.000000,0.000000
50%,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.000000,7.300000,0.000000,0.000000,1.000000,11.000000,1.698150,0.000000,1.000000,0.000000
75%,8.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,...,0.000000,7.800000,0.000000,1.000000,1.000000,12.000000,3.172850,0.000000,1.000000,0.000000
max,89.000000,104.000000,141.000000,155.000000,12.000000,8.000000,1.000000,1.000000,8.000000,1.000000,...,1.000000,10.900000,1.000000,1.000000,1.000000,18.000000,54.835098,1.000000,1.000000,1.000000


### Healthcare Visits by Medicare Recipients

From "Demand for Medical Care by the Elderly: A Finite Mixture Approach" by Partha Deb and Pravin K. Trivedi, Journal of Applied Econometrics, Vol. 12, No. 3, 1997.
>Cost and access continue to be the fundamental issues in the debate over the future of the American health-care system. An important element of this debate concerns the health-care needs of the elderly. During the past two decades, the population aged 65 and over has increased more than twice as fast as the younger population and they account for a disproportionate share of medical care expenditures.' This disproportionate share of expenditures can be attributed, in part, to the relatively greater need of the elderly for individualized care, and the increasing incidence of chronic diseases and impairment with age. Therefore, the increasing size and life expectancy of the elderly population have significant implications for health care: this motivates the present study.

We are interested in understanding the 6 separate measures of healthcare utilization provided in the NMES data. Those 6 measures are:
1. ofp  ::: visits to a physician in an office setting
2. ofnp ::: visits to a non-physician in an office setting
3. opp  ::: visits to a physician in a hospital outpatient setting
4. opnp ::: visits to a non-physician in an outpatient setting
5. emr  ::: visits to an emergency room
6. hosp ::: number of hospital stays

We will model each measure using a count data regression. We will begin by assuming the data follow a Poisson likelihood and estimate separate models for each measure. All regression models will use the same set of regressors:

In [ ]:
depvars = ['ofp','ofnp','opp','opnp','emr','hosp']

In [ ]:
test = np.array(data[['exclhlth', 'poorhlth','numchron', 'adldiff',
               'noreast','midwest','west','age',
               'black','male','married','school',
               'faminc','employed','privins','medicaid']])

In [ ]:
test

In [ ]:
x_nmes = np.hstack([np.ones((len(data),1)),np.array(data[['exclhlth', 'poorhlth','numchron', 'adldiff',
               'noreast','midwest','west','age',
               'black','male','married','school',
               'faminc','employed','privins','medicaid']
             ])])

In [ ]:
x_nmes.shape

In [ ]:
x_nmes

In [ ]:
# construct stan dictionaries
stan_dict = {}

for d in depvars:
    stan_dict[d] = {}
    stan_dict[d]['x'] = np.array(x_nmes)
    stan_dict[d]['y'] = np.array(data[d],dtype=int)
    stan_dict[d]['N'] = x_nmes.shape[0]
    stan_dict[d]['K'] = x_nmes.shape[1]

In [ ]:
stan_dict['ofp']

## Poisson regression

In [18]:
# what's in this stan model anyway?
f = open('../../stan/poisson_reg.stan', 'r')
file_contents = f.read()
print (file_contents)
f.close()

// poisson regression
data {
  int<lower=1> N;       // number of observations
  int<lower=1> K;       // number of variables/predictors/features
  matrix[N,K] x;        // matrix of explanatory variables/predictor matrix/features
  int<lower=0> y[N];    // count outcome vector
}
parameters {
  vector[K] beta;       // regression coefficients
}
model {
  // priors
  beta ~ student_t(4,0,1);

  // likelihood
  y ~ poisson_log(x * beta);
}
generated quantities {
  int y_hat[N];    // simulated values from the posterior

  // sample from poisson distribution
  for (i in 1:N) {
    y_hat[i] = poisson_log_rng(x[i] * beta);
  }
}



In [19]:
# compile Stan model
pois_model = pystan.StanModel(file="../../stan/poisson_reg.stan")

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_4d75a6a4ea19dc6e70173f79f2c6e262 NOW.


In [ ]:
# construct stan dictionaries
fit_dict = {}

for d in depvars:
    # nice message
    print('Estimating model for '+str(d)+'!')
    
    # conduct inference
    fit = pois_model.sampling(data=stan_dict[d],iter=2000,chains=4)    
    
    # store dictionary of results
    fit_dict[d] = fit.extract(permuted=True)

print('')
print('Done!')
print('')

In [ ]:
# print some pretty summary stats about the posterior
print(fit)

In [ ]:
test = np.exp(fit_dict['ofp']['beta'])

In [ ]:
fit_dict['ofp']['beta'][0,:]

In [ ]:
data['ofp'][0]

In [ ]:
y ~ poisson_log_glm(x, alpha, beta)

In [ ]:
plt.hist(data['ofp'],bins=np.arange(50))

In [ ]:
np.mean(fit_dict['hosp']['beta'],axis=0).dot(x_nmes[idx[10]])

In [ ]:
# random data points to plot
idx = np.random.choice(np.arange(len(fit_dict['hosp']['beta'])),size=16,replace=False)

# Initialise the figure and a subplot axes.
num_rows = 4
num_cols = 4
fig, ax = plt.subplots(num_rows, num_cols, figsize=(16, 16))

# overall title
fig.suptitle('Graphical depictions of the posterior',y=1.025,fontsize=18,fontfamily='serif')

c = 0
for i in np.arange(4):
    for j in np.arange(4):
        ax[i,j].hist(np.random.poisson(np.mean(fit_dict['hosp']['beta'],axis=0).dot(x_nmes[idx[c]]),size=400),bins=np.arange(15))
        ax[i,j].axvline(data['hosp'][idx[c]],color='red',lw=2)
        c += 1

# make the plot prettier
plt.tight_layout()
plt.show();      



If the outcome for a single observation y is assumed to follow a Poisson distribution, the likelihood for one observation can be written as a conditionally Poisson PMF

1y!λye−λ,

where λ=E(y|x)=g−1(η) and η=α+x⊤β is a linear predictor. For the Poisson distribution it is also true that λ=Var(y|x), i.e. the mean and variance are both λ. Later in this vignette we also show how to estimate a negative binomial regression, which relaxes this assumption of equal conditional mean and variance of y.

Because the rate parameter λ must be positive, for a Poisson GLM the link function g maps between the positive real numbers R+ (the support of λ) and the set of all real numbers R. When applied to a linear predictor η with values in R, the inverse link function g−1(η) therefore returns a positive real number.

Although other link functions are possible, the canonical link function for a Poisson GLM is the log link g(x)=ln(x). With the log link, the inverse link function is simply the exponential function and the likelihood for a single observation becomes

g−1(η)yy!e−g−1(η)=eηyy!e−eη.

## Run OLS

In [12]:
# what's in this stan model anyway?
f = open('../../stan/ols.stan', 'r')
file_contents = f.read()
print (file_contents)
f.close()

// generic ols model
data {
  // data
  int<lower=1> N;       // number of observations
  int<lower=1> K;       // number of variables/predictors/features
  matrix[N,K] x;        // matrix of explanatory variables/predictor matrix/features
  vector[N] y;          // outcome vector

  // hyperpriors for parameters (student_t)
  real mu_alpha;
  real sigma_alpha;
  real nu_alpha;
  real mu_beta;
  real sigma_beta;
  real nu_beta;
  real mu_sigma;
  real sigma_sigma;
  real nu_sigma;

  // just sample from prior?
  // if sample_prior = 1, then just sample from prior
  int<lower=0,upper=1> sample_prior;
}
parameters {
  real alpha;           // regression coefficients
  vector[K] beta;       // regression coefficients
  real<lower=0> sigma;  // standard deviation of normal distr
}
model {
  // priors
  alpha ~ student_t(nu_alpha,mu_alpha,sigma_alpha);
  beta ~ student_t(nu_beta,mu_beta,sigma_beta);
  sigma ~ student_t(nu_sigma,mu_sigma,sigma_sigma);

  // likelihood
  if (sample_prior == 0

In [13]:
# compile Stan model
pois_model = pystan.StanModel(file="../../stan/ols.stan")

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_1541e429db2e23ef47714e8ccc624e68 NOW.


CompileError: command 'x86_64-apple-darwin13.4.0-clang' failed with exit status 1

In [ ]:
# construct stan dictionaries
fit_dict = {}

for d in depvars:
    # nice message
    print('Estimating model for '+str(d)+'!')
    
    # conduct inference
    fit = pois_model.sampling(data=stan_dict[d],iter=2000,chains=4)    
    
    # store dictionary of results
    fit_dict[d] = fit.extract(permuted=True)

print('')
print('Done!')
print('')

In [ ]:
# print some pretty summary stats about the posterior
print(fit)

## WAIC on OLS

R code to compute WAIC, from [here](https://github.com/stan-dev/stan/issues/473)
```R

# The calculation of Waic!  Returns lppd, p_waic_1, p_waic_2, and waic, which we define
# as 2*(lppd - p_waic_2), as recommmended in BDA
waic <- function (stanfit){
  log_lik <- extract (stanfit, "log_lik")$log_lik
  lppd <- sum (log (colMeans(exp(log_lik))))
  p_waic_1 <- 2*sum (log(colMeans(exp(log_lik))) - colMeans(log_lik))
  p_waic_2 <- sum (colVars(log_lik))
  waic_2 <- -2*lppd + 2*p_waic_2
  return (list (waic=waic_2, p_waic=p_waic_2, lppd=lppd, p_waic_1=p_waic_1))
}
```

In [ ]:
def waic(res_dict):
    """
    Given a dictionary of Stan posterior
    draws with a loglik key, compute
    the WAIC statistic
    """
    
    